In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from columnas_seleccionadas import COLUMNAS_SELECCIONADAS

In [2]:
dataframe = pd.read_csv('Dataset_csv_unificado_completo/Totalizadores_TODO.csv')

C:\Users\franc\AppData\Local\Temp\ipykernel_1748\1861678415.py:1: DtypeWarning: Columns (124,171,196,354,355,356,357,358,365) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv('Dataset_csv_unificado_completo/Totalizadores_TODO.csv')


In [3]:
dataframe = dataframe[COLUMNAS_SELECCIONADAS]

In [4]:
dataframe.shape

(662416, 80)

# Construcción del dataset
El dataset cuenta con multiples registros sobre diferentes features relacionadas con una fabrica de cerveza que producen diferentes tipos. El dataset cuenta con (multiples) registros diarios sobre el estado de estas diferentes features, pero también con contenido basura.


In [5]:
class make_dataset():
    def __init__(self, df):
        self.df = df.copy()

    @property
    def shape(self):
        """Retorna la forma ACTUAL del dataframe interno."""
        return self.df.shape

    def columnas_problematicas(self):
        """
        Identifica, elimina y reporta columnas problemáticas:
        1. Columnas completamente vacías (todos sus valores son nulos).
        2. Columnas que contienen 'Unnamed' en su nombre (comunes en CSVs).
        3. Columnas cuyo nombre es un número (ej. "1", "2.5").
        4. Columnas cuyo nombre es un espacio en blanco (' ').
        5. Columnas de metadatos específicas (ej. 'Ultimo Dato del Dia').
        
        Retorna:
            (list): Una lista con los nombres de todas las columnas eliminadas.
        """
        
        print("--------- Buscando columnas problemáticas ---------")
        
        # --- Lógica 1: Detectar columnas TOTALMENTE NULAS ---
        columnas_totalmente_nulas = self.df.columns[self.df.isnull().all()].tolist()
        
        if columnas_totalmente_nulas:
            print(f"🧹 Detectadas {len(columnas_totalmente_nulas)} columnas totalmente nulas:")
            print(f"   -> {columnas_totalmente_nulas}")
        else:
            print("👍 No se encontraron columnas totalmente nulas.")
            
        # --- Lógica 2: Detectar columnas 'Unnamed' ---
        # Usamos str(col).lower() para ser robustos y case-insensitive
        columnas_unnamed = [col for col in self.df.columns if 'unnamed' in str(col).lower()]
        
        if columnas_unnamed:
            print(f"🧹 Detectadas {len(columnas_unnamed)} columnas 'Unnamed':")
            print(f"   -> {columnas_unnamed}")
        else:
            print("👍 No se encontraron columnas 'Unnamed'.")

        # --- Lógica 3: Detectar columnas con nombre numérico ---
        columnas_nombre_numerico = []
        for col in self.df.columns:
            try:
                # Convertir a str por si el nombre ya es un int o float
                float(str(col))
                # Si esto funciona, el nombre de la columna es un número
                columnas_nombre_numerico.append(col)
            except ValueError:
                # Si falla, el nombre no es un número (ej. "Planta (Kw)"), 
                # lo cual es correcto.
                pass
        
        if columnas_nombre_numerico:
            print(f"🧹 Detectadas {len(columnas_nombre_numerico)} columnas con nombre numérico:")
            print(f"   -> {columnas_nombre_numerico}")
        else:
            print("👍 No se encontraron columnas con nombre numérico.")

        # --- Lógica 4: Detectar columnas con nombre de espacio ---
        columnas_espacio_en_blanco = [col for col in self.df.columns if str(col) == ' ']
        
        if columnas_espacio_en_blanco:
            print(f"🧹 Detectadas {len(columnas_espacio_en_blanco)} columnas con nombre de espacio (' '):")
            print(f"   -> {columnas_espacio_en_blanco}")
        else:
            print("👍 No se encontraron columnas con nombre de espacio.")

        # --- Lógica 5: Detectar columnas de metadatos (NUEVO) ---
        columna_metadato_especifica = 'Ultimo Dato del Dia'
        columnas_metadatos = []
        if columna_metadato_especifica in self.df.columns:
            columnas_metadatos.append(columna_metadato_especifica)
            print(f"🧹 Detectada columna de metadatos: ['{columna_metadato_especifica}']")
        else:
            print("👍 No se encontró la columna de metadatos ('Ultimo Dato del Dia').")


        # --- Combinar y Eliminar ---
        # Usamos un 'set' para combinar las listas y evitar duplicados
        columnas_a_eliminar = list(set(
            columnas_totalmente_nulas + 
            columnas_unnamed + 
            columnas_nombre_numerico +
            columnas_espacio_en_blanco +
            columnas_metadatos  # <-- Añadimos la nueva lista
        ))
        
        if columnas_a_eliminar:
            print(f"\n🗑️ Eliminando un total de {len(columnas_a_eliminar)} columnas...")
            self.df.drop(columns=columnas_a_eliminar, inplace=True)
            print(f"📊 Limpieza de columnas finalizada.")
        else:
            print("\n👍 No hay columnas problemáticas para eliminar.")
        
        print("--------------------------------------------------")
        
        # Retornamos la lista de lo que se eliminó
        return columnas_a_eliminar

    # -----------------------
    # Limpieza de FILAS
    # -----------------------
    def filas_problematicas(self, columnas_clave):
        """
        Elimina y reporta filas donde columnas esenciales (como 'DIA' o 'HORA')
        tienen valores nulos.
        
        Parámetros:
            columnas_clave (list): Lista de columnas que no pueden tener nulos.
        
        Retorna:
            (int): El número total de filas eliminadas.
        """
        
        print(f"--------- Buscando filas problemáticas en {columnas_clave} ---------")
        filas_eliminadas_total = 0
        
        for col in columnas_clave:
            if col in self.df.columns:
                filas_antes = len(self.df)
                self.df.dropna(subset=[col], inplace=True)
                filas_despues = len(self.df)
                filas_eliminadas_ronda = filas_antes - filas_despues
                
                if filas_eliminadas_ronda > 0:
                    print(f"✅ Eliminadas {filas_eliminadas_ronda} filas con '{col}' vacío")
                    filas_eliminadas_total += filas_eliminadas_ronda
            
            else:
                print(f"⚠️ Aviso: La columna clave '{col}' no existe. No se limpiaron filas.")

        if filas_eliminadas_total == 0:
            print("👍 No se encontraron filas problemáticas para eliminar.")
        else:
            print(f"📊 Limpieza de filas finalizada. Total eliminadas: {filas_eliminadas_total}.")
            
        print("--------------------------------------------------")
        
        return filas_eliminadas_total
        

    # -----------------------
    # Valores negativos
    # -----------------------
    def valores_negativos_por_columna(self, columnas_permiten_negativos: list):
        """
        Identifica todas las columnas numéricas que tienen valores negativos
        y los reemplaza por np.nan.
        
        Excluye del reemplazo a las columnas especificadas en la lista
        'columnas_permiten_negativos' (ej. 'Temp Tq Intermedio').
        
        Parámetros:
            columnas_permiten_negativos (list): Lista de columnas que SÍ 
                                                pueden ser negativas.
        
        Retorna:
            (int): El número total de celdas reemplazadas.
        """
        print(f"--------- Tratando valores negativos ---------")
        
        # 1. Identificar todas las columnas numéricas del dataframe
        columnas_numericas = self.df.select_dtypes(include=['number']).columns
        
        # 2. Convertir la lista de excepciones a un 'set' para búsquedas más rápidas
        set_excepciones = set(columnas_permiten_negativos)

        total_reemplazos = 0
        columnas_afectadas = 0
        
        print(f"ℹ️ Excepciones (columnas que se ignorarán): {list(set_excepciones)}")
        
        # 3. Iterar sobre todas las columnas numéricas
        for col in columnas_numericas:
            
            # 4. Si la columna está en las excepciones, la saltamos
            if col in set_excepciones:
                continue
                
            # 5. Si no es una excepción, buscamos negativos
            
            # Creamos una máscara (un True/False) para los valores < 0
            # Usamos .loc para asegurarnos de que no haya errores de tipo
            try:
                mascara_negativos = self.df[col] < 0
            except TypeError:
                # Esto puede pasar si la columna es numérica pero tiene
                # objetos mixtos. Mejor saltarla.
                print(f"⚠️ - Omitiendo columna '{col}' (posible tipo de dato mixto).")
                continue

            # Contamos cuántos negativos hay
            negativos_count = mascara_negativos.sum()
            
            if negativos_count > 0:
                # 6. Reemplazamos esos valores por np.nan
                # Usamos .loc para modificar self.df de forma segura y directa
                self.df.loc[mascara_negativos, col] = np.nan
                
                print(f"  - Columna '{col}': {negativos_count} valores negativos convertidos a NaN.")
                total_reemplazos += negativos_count
                columnas_afectadas += 1

        if total_reemplazos == 0:
            print("👍 No se encontraron valores negativos no deseados.")
        else:
             print(f"📊 Tratamiento finalizado. {total_reemplazos} celdas reemplazadas en {columnas_afectadas} columnas.")
        
        print("--------------------------------------------------")
        
        # Retornamos el conteo por si es útil
        return total_reemplazos

    # -----------------------
    # Redondeo horarios
    # -----------------------
    def redondear_horarios(self):
        col = "HORA"
        if col not in self.df.columns:
            return self.df

        # Convertir a datetime
        self.df[col] = pd.to_datetime(self.df[col], format="%H:%M:%S", errors="coerce")

        def redondear(hora):
            if pd.isna(hora):
                return None

            # Última hora del día
            if hora.hour == 23 and hora.minute == 59 and hora.second !=59:
                return "23:59:59" #aca cambie, en teoria no deberia afectar: 

            # Redondear a la hora más cercana
            if hora.minute >= 30:
                nueva_hora = (hora + pd.Timedelta(hours=1)).replace(minute=0, second=0)
            else:
                nueva_hora = hora.replace(minute=0, second=0)

            # Si se pasa de día, dejar 23:59:59
            if nueva_hora.hour == 0 and hora.hour == 23:
                return "23:59:59"

            return nueva_hora.strftime("%H:%M:%S")

        self.df[col] = self.df[col].apply(redondear)
        return self.df

    def add_estacion(self):
      """
      Agrega una columna 'ESTACION' al dataframe con la estación del año
      (Primavera, Verano, Otoño, Invierno) según la fecha en México (hemisferio norte).
      Requiere que self.df tenga la columna 'DIA' en formato datetime.
      """
      if "DIA" not in self.df.columns:
          print("❌ No existe la columna 'DIA' en el dataframe.")
          return

      # Asegurar formato datetime
      self.df["DIA"] = pd.to_datetime(self.df["DIA"], errors="coerce")

      def obtener_estacion(fecha):
          if pd.isna(fecha):
              return None

          año = fecha.year
          # Fechas de cambio de estación (en hemisferio norte)
          primavera = (pd.Timestamp(year=año, month=3, day=21), pd.Timestamp(year=año, month=6, day=20))
          verano = (pd.Timestamp(year=año, month=6, day=21), pd.Timestamp(year=año, month=9, day=22))
          otonio = (pd.Timestamp(year=año, month=9, day=23), pd.Timestamp(year=año, month=12, day=20))
          # Invierno cruza el año
          invierno_1 = (pd.Timestamp(year=año, month=12, day=21), pd.Timestamp(year=año + 1, month=3, day=20))

          if primavera[0] <= fecha <= primavera[1]:
              return "Primavera"
          elif verano[0] <= fecha <= verano[1]:
              return "Verano"
          elif otonio[0] <= fecha <= otonio[1]:
              return "Otoño"
          elif (fecha >= invierno_1[0]) or (fecha <= invierno_1[1]):
              return "Invierno"
          else:
              return None

      self.df["ESTACION"] = self.df["DIA"].apply(obtener_estacion)
      print("✅ Columna 'ESTACION' agregada correctamente.")

    # -----------------------
    # Agregar temperatura y día de la semana
    # -----------------------
    def add_temp_y_dia(self, temp_df):
    # Columnas fecha
      if 'DIA' not in self.df.columns or 'fecha' not in temp_df.columns:
          return

      self.df["DIA"] = pd.to_datetime(self.df["DIA"], format="%Y-%m-%d", errors="coerce")
      temp_df["fecha"] = pd.to_datetime(temp_df["fecha"], format="%Y-%m-%d", errors="coerce")

      # Columna combinada fecha-hora
      self.df["fecha_hora"] = pd.to_datetime(
          self.df["DIA"].astype(str) + " " + self.df["HORA"].astype(str),
          format="%Y-%m-%d %H:%M:%S",
          errors="coerce"
      )
      temp_df["fecha_hora"] = pd.to_datetime(
          temp_df["fecha"].astype(str) + " " + temp_df["hora"].astype(str),
          format="%Y-%m-%d %H:%M:%S",
          errors="coerce"
      )

      # Caso especial 23:59:59 → día siguiente 00:00:00
      mask_2359 = self.df["HORA"] == "23:59:59"
      self.df.loc[mask_2359, "fecha_hora"] = self.df.loc[mask_2359, "fecha_hora"] + pd.Timedelta(days=1)
      self.df.loc[mask_2359, "fecha_hora"] = self.df.loc[mask_2359, "fecha_hora"].dt.floor("D")

      # Merge para traer temperatura horaria
      self.df = pd.merge(
          self.df,
          temp_df[["fecha_hora", "temperature_2m"]],
          on="fecha_hora",
          how="left"
      )

      # Limpiar columna auxiliar
      self.df.drop(columns=["fecha_hora"], inplace=True)

      # Agregar día de la semana
      if "DIA" in self.df.columns:
          self.df["DIA_SEMANA"] = self.df["DIA"].dt.day_name()

      # --- NUEVO: agregar temperatura promedio por día ---
      temp_promedio = temp_df.groupby("fecha")["temperature_2m"].mean().reset_index()
      temp_promedio.rename(columns={"temperature_2m": "temperature_promedio_dia"}, inplace=True)

      self.df = pd.merge(
          self.df,
          temp_promedio,
          left_on="DIA",
          right_on="fecha",
          how="left"
      )
      # Limpiar columna auxiliar
      self.df.drop(columns=["fecha"], inplace=True)


    # -----------------------
    # Promedio de EE Frio / Hl por hora
    # -----------------------
    def promedio_frio_por_hora(self):
        if "HORA" not in self.df.columns or "EE Frio / Hl" not in self.df.columns:
            return pd.DataFrame()

        self.df["HORA"] = pd.to_datetime(self.df["HORA"], format="%H:%M:%S", errors="coerce").dt.time
        resumen = (
            self.df.groupby("HORA", as_index=False)["EE Frio / Hl"]
            .mean()
            .rename(columns={"EE Frio / Hl": "frio_promedio"})
            .sort_values("HORA")
        )
        return resumen
    
    def verificar_duplicado(self, col1, col2):
        """
        Verifica si dos columnas del DataFrame interno (self.df) son duplicadas.
        
        Casos que comprueba:
        1. Duplicado Perfecto (idénticas, incluyendo NaNs).
        2. Duplicado con NaNs (idénticas en todos los valores no-nulos).
        3. No duplicadas.
        """
        
        print(f"\n--- Verificando: '{col1}' vs '{col2}' ---")
        
        # --- 1. Chequear existencia ---
        columnas_faltantes = []
        if col1 not in self.df.columns:
            columnas_faltantes.append(col1)
        if col2 not in self.df.columns:
            columnas_faltantes.append(col2)
            
        if columnas_faltantes:
            print(f"  ❌ Error: Las siguientes columnas no se encontraron: {columnas_faltantes}.")
            print("     Por favor, revisa que los nombres sean exactos (mayúsculas, espacios, etc.)")
            return

        # --- 2. Test 1: Duplicado Perfecto (con .equals()) ---
        if self.df[col1].equals(self.df[col2]):
            print("  ✅ Resultado: IDÉNTICAS (Perfect match).")
            print(f"     (Puedes eliminar una, por ejemplo '{col2}')")
            return

        # --- 3. Test 2: Duplicado con diferencia de NaNs ---
        mask_non_null = self.df[col1].notna() & self.df[col2].notna()
        
        if mask_non_null.sum() == 0:
            print("  ⚠️ Resultado: No se pueden comparar (nunca tienen valores al mismo tiempo).")
            return
            
        diferencias = (self.df.loc[mask_non_null, col1] != self.df.loc[mask_non_null, col2])
        num_diferencias = diferencias.sum()

        if num_diferencias == 0:
            print("  ⚠️ Resultado: DUPLICADAS (Difieren solo en la posición de los NaNs).")
            print("     (Son idénticas en todas las filas donde ambas tienen datos).")
        else:
            print("  ❌ Resultado: NO SON DUPLICADAS.")
            print(f"     (Tienen {num_diferencias} valores diferentes donde ambas tienen datos).")

        nan_1 = self.df[col1].isna().sum()
        nan_2 = self.df[col2].isna().sum()
        print(f"     Contexto: '{col1}' tiene {nan_1} NaNs. '{col2}' tiene {nan_2} NaNs.")


    def verificar_relevo(self, col1, col2, assume_sorted_index=True):
        """
        Verifica si una columna 'toma el relevo' de otra en self.df.
        
        Retorna True si se cumplen dos condiciones:
        1. Ambas columnas tienen *algunos* datos (no están 100% vacías).
        2. NUNCA tienen datos en la misma fila (no hay superposición).
        """
        
        print(f"\n--- Verificando relevo: '{col1}' vs '{col2}' ---")
        
        # --- 1. Chequear existencia ---
        if col1 not in self.df.columns or col2 not in self.df.columns:
            print(f"  ❌ Error: Una o ambas columnas no se encontraron.")
            return False

        # --- 2. Obtener máscaras de datos válidos (no-NaN) ---
        validos_col1 = self.df[col1].notna()
        validos_col2 = self.df[col2].notna()

        # --- 3. Condición 1: ¿Ambas columnas tienen *algunos* datos? ---
        ambas_tienen_datos = validos_col1.any() and validos_col2.any()
        
        if not ambas_tienen_datos:
            print("  ❌ FALSO: Una o ambas columnas están 100% vacías (NaN).")
            return False

        # --- 4. Condición 2: ¿Hay superposición? (Debe ser 0) ---
        superposicion = (validos_col1 & validos_col2).sum()
        
        if superposicion > 0:
            print(f"  ❌ FALSO: Las columnas se superponen en {superposicion} filas.")
            return False

        # Si llegamos aquí, ambas condiciones se cumplen.
        print("  ✅ VERDADERO: Las columnas tienen datos pero nunca se superponen.")
        print("     (Esto confirma el patrón de 'relevo' de datos).")
        
        # --- 5. (Opcional) Proveer más contexto si el índice está ordenado ---
        if assume_sorted_index:
            try:
                fin_col1 = self.df.index[validos_col1].max()
                inicio_col2 = self.df.index[validos_col2].min()
                fin_col2 = self.df.index[validos_col2].max()
                inicio_col1 = self.df.index[validos_col1].min()

                if fin_col1 < inicio_col2:
                    print(f"     (Patrón limpio: '{col1}' termina en {fin_col1}, luego '{col2}' empieza en {inicio_col2})")
                elif fin_col2 < inicio_col1:
                    print(f"     (Patrón limpio: '{col2}' termina en {fin_col2}, luego '{col1}' empieza en {inicio_col1})")
                else:
                    print("     (Los datos están intercalados, no en un bloque 'antes' y 'después')")
            
            except Exception as e:
                print(f"     (No se pudo verificar el orden cronológico del relevo: {e})")
                
        return True
    
    # -----------------------
    # Verificación de Consistencia
    # -----------------------

    def verificar_consistencia_binaria(self, columnas_binarias: list):
        """
        Verifica si hay inconsistencias en columnas binarias para tuplas (DIA, HORA) duplicadas.

        Una inconsistencia ocurre si para el mismo (DIA, HORA), una columna binaria
        tiene valores diferentes (ej. 0 y 1), ignorando los NaN.
        
        Args:
            columnas_binarias (list): La lista de columnas binarias a verificar.
            
        Returns:
            tuple: (bool, dict)
                - bool: True si se encontraron inconsistencias, False si no.
                - dict: Un diccionario donde las claves son las tuplas (DIA, HORA)
                        problemáticas y los valores son las columnas con
                        inconsistencias.
        """
        
        print("--- Iniciando verificación de consistencia en tuplas (DIA, HORA) duplicadas ---")
        
        # 1. Agrupar por la tupla clave
        # Usamos el DataFrame interno 'self.df'
        grupos = self.df.groupby(['DIA', 'HORA'])
        
        problematic_groups = {}
        hay_inconsistencia = False
        
        grupos_con_duplicados = 0
        
        # 2. Iterar sobre cada grupo (DIA, HORA)
        for (dia, hora), group in grupos:
            
            # Solo nos interesan los grupos con duplicados
            if len(group) > 1:
                grupos_con_duplicados += 1
                inconsistencias_en_col = []
                
                # 3. Revisar cada columna binaria dentro de ese grupo
                for col in columnas_binarias:
                    if col not in group.columns:
                        continue # Seguridad por si la columna no existe
                    
                    # 4. Obtener los valores únicos NO-NAN
                    # Esta es la lógica clave: ignoramos los NaN
                    valores_unicos = group[col].dropna().unique()
                    
                    # 5. Si hay más de 1 valor único (ej. [0.0, 1.0]), es una inconsistencia
                    if len(valores_unicos) > 1:
                        hay_inconsistencia = True
                        inconsistencias_en_col.append(col)
                
                # 6. Si encontramos columnas inconsistentes, guardamos el reporte
                if inconsistencias_en_col:
                    problematic_groups[(dia, hora)] = inconsistencias_en_col

        print(f"\nRevisados {len(grupos)} grupos ('DIA', 'HORA') únicos.")
        print(f"Se encontraron {grupos_con_duplicados} grupos con filas duplicadas.")
        
        if hay_inconsistencia:
            print(f"❌ ¡Problema! Se encontraron {len(problematic_groups)} tuplas (DIA, HORA) con datos binarios inconsistentes.")
        else:
            print("✅ ¡Éxito! Todas las tuplas (DIA, HORA) duplicadas son consistentes en las columnas binarias.")
            
        return hay_inconsistencia, problematic_groups
    
    def verificar_suma_jerarquica(self, hipotesis_dict):
        """
        Verifica si columnas "Totales" son la suma de sus "partes".
        
        Recibe un diccionario donde cada clave es la columna "Total" y
        cada valor es una lista de las columnas "Parte".
        
        Args:
            hipotesis_dict (dict): Diccionario de hipótesis. 
                                   Ej: {'Total': ['Parte1', 'Parte2']}
        """
        
        print(f"--- Iniciando verificación de {len(hipotesis_dict)} sumas jerárquicas ---")
        
        for col_total, lista_partes in hipotesis_dict.items():
            print(f"\n--- Verificando: '{col_total}' vs Suma de {len(lista_partes)} partes ---")
            
            # --- 1. Verificar que todas las columnas existan ---
            columnas_a_chequear = [col_total] + lista_partes
            columnas_faltantes = [col for col in columnas_a_chequear if col not in self.df.columns]
            
            if columnas_faltantes:
                print(f"  ❌ Error: Faltan columnas: {columnas_faltantes}. Saltando.")
                continue
                
            # --- 2. Sumamos las partes ---
            # fillna(0) es crucial por si una de las partes tiene NaN
            suma_calculada = self.df[lista_partes].fillna(0).sum(axis=1)
            
            # --- 3. Comparamos el total reportado vs. la suma calculada ---
            # Usamos np.allclose para comparar números flotantes, lo que
            # permite pequeños errores de precisión (tolerancia).
            son_iguales = np.allclose(
                self.df[col_total].fillna(0),  # Rellenamos NaNs en el total
                suma_calculada, 
                rtol=1e-03, # Tolerancia relativa (0.1%)
                atol=1e-05  # Tolerancia absoluta
            )

            if son_iguales:
                print(f"  ✅ Resultado: REDUNDANCIA CONFIRMADA.")
                print(f"     ('{col_total}' es la suma de sus partes).")
            else:
                print(f"  ❌ Resultado: NO SON REDUNDANTES.")
                print(f"     ('{col_total}' NO es la suma simple de sus partes).")
                
                # Mostramos un resumen de la diferencia
                try:
                    diferencia = (self.df[col_total] - suma_calculada).abs()
                    print(f"     Diferencia promedio: {diferencia.mean():.4f}")
                    print(f"     Diferencia máxima:   {diferencia.max():.4f}")
                except TypeError:
                    print("     (No se pudo calcular la diferencia, posible error de tipos).")

    def verificar_exclusividad_mutua(self, hipotesis_grupos):
        """
        Verifica si grupos de columnas binarias (dummies) son mutuamente excluyentes.

        La prueba comprueba que para cualquier fila, la suma de las columnas
        del grupo sea como máximo 1 (es decir, solo una puede estar "activa").
        
        Args:
            hipotesis_grupos (dict): Diccionario de hipótesis.
                                   Ej: {'Grupo Mostos': ['Mosto1', 'Mosto2']}
        """
        
        print(f"--- Iniciando verificación de {len(hipotesis_grupos)} grupos mutuamente excluyentes ---")

        for nombre_grupo, lista_columnas in hipotesis_grupos.items():
            print(f"\n--- Verificando Grupo: '{nombre_grupo}' ({len(lista_columnas)} columnas) ---")

            # --- 1. Verificar que todas las columnas existan ---
            columnas_faltantes = [col for col in lista_columnas if col not in self.df.columns]
            
            if columnas_faltantes:
                print(f"  ❌ Error: Faltan columnas: {columnas_faltantes}. Saltando.")
                continue
            
            # --- 2. Calcular la suma por fila (asumiendo NaN=0) ---
            try:
                # fillna(0) es clave para tratar los NaNs como "inactivo"
                suma_por_fila = self.df[lista_columnas].fillna(0).sum(axis=1)
            except TypeError:
                print(f"  ❌ Error: No se pudo sumar. ¿Alguna columna no es numérica? Saltando.")
                # Esto puede pasar si las columnas están mal identificadas (ej. 'Totalizador Bba P1' 
                # que salió como 'object' pero se quiere tratar como binaria)
                continue

            # --- 3. Analizar el resultado ---
            max_suma = suma_por_fila.max()
            
            if max_suma <= 1:
                print(f"  ✅ Resultado: SON MUTUAMENTE EXCLUYENTES.")
                print(f"     (Nunca hay más de una columna 'activa' (1) al mismo tiempo).")
            else:
                print(f"  ❌ Resultado: NO SON MUTUAMENTE EXCLUYENTES.")
                filas_violadas = (suma_por_fila > 1).sum()
                print(f"     (Se encontraron {filas_violadas} filas donde la suma es > 1).")
                print(f"     (La suma máxima encontrada en una fila fue: {max_suma}).")

    def forzar_conversion_numerica(self, columnas_a_convertir: list):
        """
        Fuerza la conversión de una lista de columnas a tipo numérico.
        
        Utiliza pd.to_numeric con errors='coerce', que convertirá 
        automáticamente cualquier valor no numérico (ej. "Error") en NaN.
        
        Args:
            columnas_a_convertir (list): Lista de nombres de columnas.
            
        Returns:
            int: El número de columnas que cambiaron su tipo de dato.
        """
        
        print(f"--- Forzando conversión a numérico en {len(columnas_a_convertir)} columnas ---")
        
        columnas_convertidas = 0
        for col in columnas_a_convertir:
            if col in self.df.columns:
                tipo_anterior = self.df[col].dtype
                
                # Aplicamos la conversión robusta
                self.df[col] = pd.to_numeric(self.df[col], errors='coerce')
                
                tipo_nuevo = self.df[col].dtype
                
                if tipo_anterior != tipo_nuevo:
                    print(f"  ✅ Columna '{col}': Convertida ({tipo_anterior} -> {tipo_nuevo})")
                    columnas_convertidas += 1
                else:
                    print(f"  ℹ️ Columna '{col}': Ya era numérica ({tipo_nuevo}).")
                    
            else:
                print(f"  ❌ Error: No se encontró la columna '{col}'.")

        print(f"\nConversión completada. Se convirtieron {columnas_convertidas} columnas.")
        return columnas_convertidas
    

    # -----------------------
    # Agrupación y Consolidación
    # -----------------------

    def agrupar_por_media(self, columnas_binarias: list, columnas_texto: list = None):
        """
        [MODIFICADO] Agrupa el DataFrame interno (self.df) por (DIA) para 
        consolidar todas las filas en un PROMEDIO DIARIO.
        
        ¡ADVERTENCIA: Esto elimina la resolución horaria!
        
        - Usa 'mean' (promedio) para columnas numéricas.
        - Usa 'max' para columnas binarias/dummies (si hubo un 1 en el día, queda 1).
        - Usa 'first' para columnas de texto.
        
        El DataFrame self.df se actualiza internamente.
        
        Args:
            columnas_binarias (list): Lista de nombres de las columnas binarias.
            columnas_texto (list, optional): Lista de otras columnas de texto 
                                              (aparte de DIA) que deban mantenerse.
        """
        
        print("--- Iniciando agrupación por (DIA) usando la media (Promedio Diario) ---")
        
        # 1. Definir las columnas clave (MODIFICADO)
        columnas_clave = ['DIA'] # Agrupamos solo por DÍA
        
        # 2. Crear el diccionario de agregación
        agg_dict = {}
        
        # Convertir listas a 'sets' para búsquedas rápidas
        set_binarias = set(columnas_binarias)
        set_texto = set(columnas_texto) if columnas_texto else set()
        
        filas_antes = self.df.shape[0]

        # 3. Iterar sobre todas las columnas para construir el diccionario
        for col in self.df.columns:
            # Ignorar las columnas clave
            if col in columnas_clave:
                continue
            
            # Ignorar HORA explícitamente, ya que la estamos perdiendo
            if col == 'HORA':
                continue
                
            # Si es una columna binaria...
            if col in set_binarias:
                agg_dict[col] = 'max'
            
            # Si es una columna de texto...
            elif col in set_texto:
                agg_dict[col] = 'first'
            
            # Si es una columna numérica (no-binaria y no-clave)...
            elif pd.api.types.is_numeric_dtype(self.df[col]):
                agg_dict[col] = 'mean'
            
            # Si es cualquier otra cosa (texto no especificado)...
            else:
                agg_dict[col] = 'first'
                
        # 4. Ejecutar la agregación
        if not agg_dict:
            print("No hay columnas para agrupar. Abortando.")
            return

        print(f"Agrupando {len(agg_dict)} columnas por (DIA)...")
        
        # Agrupamos y reseteamos el índice
        df_agrupado = self.df.groupby(columnas_clave).agg(agg_dict).reset_index()
        
        # 5. Actualizar el DataFrame interno
        self.df = df_agrupado
        filas_despues = self.df.shape[0]
        
        print("\n✅ Agrupación Diaria completada.")
        print(f"   Filas reducidas de {filas_antes} a {filas_despues}.")
    


Vamos a eliminar columnas problematicas. Para ello consideraremos:
- Columnas con todos sus valores nulls o bien alguna alternativa en string ("none", "null") etc
- COlumnas probleamticas (nombres indefinidos como unnamed, .numero, ' ')

Mismo analisis para las filas, donde consideraremos:
- Filas donde datos claves como dia/hora son nulos

In [6]:
obj = make_dataset(dataframe)

columnas_eliminadas = obj.columnas_problematicas()
filas_eliminadas = obj.filas_problematicas(['DIA', 'HORA'])

print(dataframe.shape)
print(obj.shape)




--------- Buscando columnas problemáticas ---------
👍 No se encontraron columnas totalmente nulas.
👍 No se encontraron columnas 'Unnamed'.
👍 No se encontraron columnas con nombre numérico.
👍 No se encontraron columnas con nombre de espacio.
👍 No se encontró la columna de metadatos ('Ultimo Dato del Dia').

👍 No hay columnas problemáticas para eliminar.
--------------------------------------------------
--------- Buscando filas problemáticas en ['DIA', 'HORA'] ---------
✅ Eliminadas 3618 filas con 'DIA' vacío
✅ Eliminadas 1 filas con 'HORA' vacío
📊 Limpieza de filas finalizada. Total eliminadas: 3619.
--------------------------------------------------
(662416, 80)
(658797, 80)


**Valores negativos**

Filas con valores negativos: Hay muchas filas con valores negativos. Para ello nos ponemos a inspeccionar si alguna de estas es lógica o probable que alberguen valores negativos y no se tratan de errores.
    - Variables de temperatura en centigrados pueden ser negativas
    - las variables totalizadoras NO pueden ser negativas
    - columnas asociadas a tasas de volumen (/hl) no deberian ser negativas
    - Columnas totalizadoras no tienen que ser negativas
    - Columnas de potencia en Kw no deberían ser negativas.
    - Tot Vap Paset L3/hora y vapor de caldera deben ser positivos
    - Retorno PLanta C=2. Es posible que sea negativo
    - Red paste L3 y Agua lavadora L3. flujos probablemente positivos

In [7]:

columnas_excepcion = [
    'Temp Tq Intermedio',
    'Retorno Planta CO2'
]
total_reemplazos = obj.valores_negativos_por_columna(columnas_excepcion)

--------- Tratando valores negativos ---------
ℹ️ Excepciones (columnas que se ignorarán): ['Retorno Planta CO2', 'Temp Tq Intermedio']
  - Columna 'EE Planta / Hl': 28 valores negativos convertidos a NaN.
  - Columna 'EE Elaboracion / Hl': 93 valores negativos convertidos a NaN.
  - Columna 'EE Bodega / Hl': 295 valores negativos convertidos a NaN.
  - Columna 'EE Servicios / Hl': 4 valores negativos convertidos a NaN.
  - Columna 'EE Aire / Hl': 2 valores negativos convertidos a NaN.
  - Columna 'EE Agua / Hl': 193 valores negativos convertidos a NaN.
  - Columna 'Agua Planta / Hl': 36 valores negativos convertidos a NaN.
  - Columna 'Agua Planta de Agua/Hl': 656 valores negativos convertidos a NaN.
  - Columna 'Produccion Agua / Hl': 38 valores negativos convertidos a NaN.
  - Columna 'CO 2 / Hl': 2 valores negativos convertidos a NaN.
  - Columna 'Planta (Kw)': 31 valores negativos convertidos a NaN.
  - Columna 'Elaboracion (Kw)': 177 valores negativos convertidos a NaN.
  - Colum

**Columnas duplicadas**
Vamos a inspeccionar si hay algunas columnas repetidas. 
Las hipotesis que planteamos son las siguientes:
- hl de mosto y hl de mosto copia. Si resultaron ser iguales
- tot l3 la4 y planta de c02 vs tot l3. l4 y planta de co2: 
- planta(kw) y KW gral planta: No son iguales.


In [8]:


# 2. Lista de hipótesis a probar
hipotesis_duplicados = [
    ('Hl de Mosto', 'Hl de Mosto Copia'),
    ('Tot L3, L4 y Planta de CO2', 'Tot L3. L4 y Planta de CO2'),
    ('Planta (Kw)', 'KW Gral Planta')
]

# --- Sección 1: Verificación de Duplicados ---
print("======================================================")
print("--- 1. Verificando Hipótesis de Duplicados ---")
print("======================================================")

for col1, col2 in hipotesis_duplicados:
    # Llamamos al MÉTODO de la clase
    obj.verificar_duplicado(col1, col2)

# --- Sección 2: Verificación de Relevo (Handoff) ---
print("\n\n======================================================")
print("--- 2. Verificando Hipótesis de Relevo de Datos ---")
print("======================================================")

col_relevo_1 = 'Tot L3, L4 y Planta de CO2'
col_relevo_2 = 'Tot L3. L4 y Planta de CO2'

# Llamamos al MÉTODO de la clase y guardamos el resultado
hubo_relevo = obj.verificar_relevo(
    col_relevo_1, 
    col_relevo_2, 
    assume_sorted_index=True
)

# --- Sección 3: Unificación basada en el resultado de Relevo ---
if hubo_relevo:
    print("\n--- Unificando columnas con relevo... ---")
    
    col_unificada = 'Tot_L3_L4_CO2_Unificado' # Nombre nuevo y limpio

    # 2. Verificar que existan antes de unificar (aunque 'verificar_relevo' ya lo hizo)
    if col_relevo_1 in obj.df.columns and col_relevo_2 in obj.df.columns:
        
        # 3. Crear la columna unificada
        # Rellena los NaN de la columna 1 con los valores de la columna 2
        obj.df[col_unificada] = obj.df[col_relevo_1].fillna(obj.df[col_relevo_2])

        # 4. Eliminar las columnas originales
        obj.df.drop(columns=[col_relevo_1, col_relevo_2], inplace=True)
        
        print(f"✅ Columnas unificadas exitosamente en '{col_unificada}'.")
        print(f"   Columnas originales eliminadas.")
        
    else:
        # Esto no debería pasar si hubo_relevo es True, pero es una buena práctica
        print(f"❌ Error: No se encontraron las columnas '{col_relevo_1}' o '{col_relevo_2}' para unificar.")
else:
     print(f"\n--- No se detectó relevo para '{col_relevo_1}', no se unificarán. ---")


print("\n======================================================")
print("--- Verificación y unificación completadas ---")
print("======================================================")

--- 1. Verificando Hipótesis de Duplicados ---

--- Verificando: 'Hl de Mosto' vs 'Hl de Mosto Copia' ---
  ❌ Error: Las siguientes columnas no se encontraron: ['Hl de Mosto Copia'].
     Por favor, revisa que los nombres sean exactos (mayúsculas, espacios, etc.)

--- Verificando: 'Tot L3, L4 y Planta de CO2' vs 'Tot L3. L4 y Planta de CO2' ---
  ❌ Error: Las siguientes columnas no se encontraron: ['Tot L3, L4 y Planta de CO2'].
     Por favor, revisa que los nombres sean exactos (mayúsculas, espacios, etc.)

--- Verificando: 'Planta (Kw)' vs 'KW Gral Planta' ---
  ❌ Error: Las siguientes columnas no se encontraron: ['KW Gral Planta'].
     Por favor, revisa que los nombres sean exactos (mayúsculas, espacios, etc.)


--- 2. Verificando Hipótesis de Relevo de Datos ---

--- Verificando relevo: 'Tot L3, L4 y Planta de CO2' vs 'Tot L3. L4 y Planta de CO2' ---
  ❌ Error: Una o ambas columnas no se encontraron.

--- No se detectó relevo para 'Tot L3, L4 y Planta de CO2', no se unificarán. -

**columnas compuestas**
Hay columnas que puedens ser la suma de dos columnas: 
A su vez analizaremos si hay columnas que son suma de columnas. Las hipotesis:
- M3_tot_gas vs TOT GAS
- KW linea 3 y 4 quisas sean la suma de KW linea 3 + kw linea4
- Hl mosto = hl mosto + hl budweiser + hlmoste

In [9]:
# ======================================================
# --- EJECUCIÓN DEL SCRIPT DE VERIFICACIÓN ---
# ======================================================

# --- Sección 3: Verificación de Sumas Jerárquicas (NUEVO) ---
print("\n\n======================================================")
print("--- 3. Verificando Hipótesis de Sumas Jerárquicas ---")
print("======================================================")

# Definimos el diccionario de hipótesis
hipotesis_sumas = {
    
    'KW Linea 3 y 4': [
        'KW Linea 3', 
        'KW Linea 4'
    ],
    
    'Planta (Kw)': [
        'Elaboracion (Kw)', 
        'Bodega (Kw)', 
        'Cocina (Kw)', 
        'Envasado (Kw)', 
        'Servicios (Kw)', 
        'Restos Planta (Kw)'
    ],
    
    'Servicios (Kw)': [
         'Sala Maq (Kw)', 
         'Aire (Kw)', 
         'Calderas (Kw)', 
         'Efluentes (Kw)', 
         'Frio (Kw)', 
         'Pta Agua / Eflu (Kw)', 
         'Prod Agua (Kw)', 
         'Resto Serv (Kw)'
    ]
}

# Llamamos al nuevo método de la clase
obj.verificar_suma_jerarquica(hipotesis_sumas)

print("\n======================================================")
print("--- Verificación completada ---")
print("======================================================")

#como no verificaron ser sumas, damos por hecho que no son redundantes



--- 3. Verificando Hipótesis de Sumas Jerárquicas ---
--- Iniciando verificación de 3 sumas jerárquicas ---

--- Verificando: 'KW Linea 3 y 4' vs Suma de 2 partes ---
  ❌ Error: Faltan columnas: ['KW Linea 3 y 4', 'KW Linea 3', 'KW Linea 4']. Saltando.

--- Verificando: 'Planta (Kw)' vs Suma de 6 partes ---
  ❌ Error: Faltan columnas: ['Restos Planta (Kw)']. Saltando.

--- Verificando: 'Servicios (Kw)' vs Suma de 8 partes ---
  ❌ Error: Faltan columnas: ['Sala Maq (Kw)', 'Pta Agua / Eflu (Kw)', 'Resto Serv (Kw)']. Saltando.

--- Verificación completada ---


**Columnas categoricas**
Vamos a inspeccionar las columnas categoricas. 
- Buscamos las columnas que son del tipo 'Objetct' o 'Category'
- Que solamente tienen 1 y 0 de valores posibles. Esperamos encontrar aca la categoria de cerveza.

dentro de las columnas objetos:
- muchas son numericas, las cambiamos

Dentro de las columnas binarias:
- tenemos dummies como mosto, combustible, vapor
- banderas de estados



In [10]:

# --- 1. Columnas categóricas (texto, objeto) ---
cols_categoricas_texto = obj.df.select_dtypes(include=['object', 'category']).columns.tolist()

# --- 2. Columnas binarias (numéricas 0/1) ---
print("Iniciando búsqueda eficiente de columnas binarias...")
cols_binarias_numericas = []

# Bucle eficiente (LA SOLUCIÓN):
# Iteramos sobre todos los nombres de columnas (una lista ligera)
for col in obj.df.columns:
    
    # Verificamos el tipo de la columna SIN copiarla
    if pd.api.types.is_numeric_dtype(obj.df[col]):
        
        # Si es numérica, aplicamos tu lógica original
        # .unique() es rápido en columnas individuales
        try:
            valores_unicos = obj.df[col].dropna().unique()
            
            if set(valores_unicos).issubset({0, 1}):
                cols_binarias_numericas.append(col)
        except Exception as e:
            print(f"  - Advertencia: No se pudo procesar la columna '{col}'. Error: {e}")

print("Búsqueda de binarias completada.")

# --- 3. Combinar ambas listas ---
columnas_finales = list(set(cols_categoricas_texto + cols_binarias_numericas))

print("\nColumnas categóricas o binarias (0/1):")
print(f" Categorias texto: {cols_categoricas_texto}")
print(f" Binarias numéricas: {cols_binarias_numericas}")


# ======================================================
# --- EJECUCIÓN DEL SCRIPT DE VERIFICACIÓN ---
# ======================================================

print("\n\n======================================================")
print("--- 4. Verificando Hipótesis de Exclusividad Mutua (Dummies) ---")
print("======================================================")

# Definimos el diccionario de hipótesis
hipotesis_dummies = {
    
    'Grupo Mostos': [
        'HL Mosto Budweiser', 'HL Mosto Local', 'HL Mosto Fuerte', 
        'HL Mosto Indio', 'HL Mosto Palermo', 'HL Mosto Bieckert', 
        'HL Mosto Malta', 'HL Mosto Frost', 'Hl Session IPA', 'Hl Reserva 8'
    ],
    
    'Grupo Fuel Oil Tanks': [
        'Fuel Oil Tk1 (Kg)', 
        'Fuel Oil Tk2 (Kg)'
    ],
    
    'Grupo Calderas (Vapor)': [
         'Tot_Vapor_Caldera 3',
         'VAPOR DE CALDERA 1 KG'
    ], 
    'Grupo Reservas': [
        'Hl Reserva 7', 
        'Hl Reserva 8'
    ]
}

# Llamamos al nuevo método de la clase
# (Asegúrate de que las columnas en hipotesis_dummies existan 
# y se hayan identificado correctamente como binarias)
obj.verificar_exclusividad_mutua(hipotesis_dummies)

print("\n======================================================")
print("--- Verificación completada ---")
print("======================================================")

Iniciando búsqueda eficiente de columnas binarias...


Búsqueda de binarias completada.

Columnas categóricas o binarias (0/1):
 Categorias texto: ['DIA', 'HORA']
 Binarias numéricas: []


--- 4. Verificando Hipótesis de Exclusividad Mutua (Dummies) ---
--- Iniciando verificación de 4 grupos mutuamente excluyentes ---

--- Verificando Grupo: 'Grupo Mostos' (10 columnas) ---
  ❌ Error: Faltan columnas: ['HL Mosto Budweiser', 'HL Mosto Local', 'HL Mosto Fuerte', 'HL Mosto Indio', 'HL Mosto Palermo', 'HL Mosto Bieckert', 'HL Mosto Malta', 'HL Mosto Frost', 'Hl Session IPA', 'Hl Reserva 8']. Saltando.

--- Verificando Grupo: 'Grupo Fuel Oil Tanks' (2 columnas) ---
  ❌ Error: Faltan columnas: ['Fuel Oil Tk1 (Kg)', 'Fuel Oil Tk2 (Kg)']. Saltando.

--- Verificando Grupo: 'Grupo Calderas (Vapor)' (2 columnas) ---
  ❌ Error: Faltan columnas: ['Tot_Vapor_Caldera 3', 'VAPOR DE CALDERA 1 KG']. Saltando.

--- Verificando Grupo: 'Grupo Reservas' (2 columnas) ---
  ❌ Error: Faltan columnas: ['Hl Reserva 7', 'Hl Reserva 8']. Saltando.

--- Verificación c

In [11]:
# --- 1. Script de Diagnóstico (el que ya tenías) ---
columnas_sospechosas = [
    'Nivel Silo Bagazo Norte (1)', 
    'KW Trafo 8', 
    'Totalizador Bba P1', 
    'Totalizador Bba P2', 
    'Totalizador Bba P4', 
    'Totalizador Bba Envasado', 
    'Totalizador Bba P51'
]

print("--- Revisando valores únicos de columnas 'texto' sospechosas ---")

for col in columnas_sospechosas:
    if col in obj.df.columns:
        # Mostramos los primeros 20 valores únicos
        print(f"\nValores en '{col}':")
        # Asegurarnos de manejar el error si hay menos de 20 únicos
        try:
            print(obj.df[col].unique()[:20])
        except IndexError:
            print(obj.df[col].unique())

print("\n" + "="*50 + "\n")

# --- 2. Llamada al nuevo método (la "instanciación") ---
# Ahora que viste los valores, llamas al método de tu objeto 'obj'
# para que realice la conversión en su dataframe interno (obj.df)

obj.forzar_conversion_numerica(columnas_sospechosas)

--- Revisando valores únicos de columnas 'texto' sospechosas ---


--- Forzando conversión a numérico en 7 columnas ---
  ❌ Error: No se encontró la columna 'Nivel Silo Bagazo Norte (1)'.
  ❌ Error: No se encontró la columna 'KW Trafo 8'.
  ❌ Error: No se encontró la columna 'Totalizador Bba P1'.
  ❌ Error: No se encontró la columna 'Totalizador Bba P2'.
  ❌ Error: No se encontró la columna 'Totalizador Bba P4'.
  ❌ Error: No se encontró la columna 'Totalizador Bba Envasado'.
  ❌ Error: No se encontró la columna 'Totalizador Bba P51'.

Conversión completada. Se convirtieron 0 columnas.


0

In [12]:
print(list(obj.df.columns))
print(obj.df.shape)

['DIA', 'HORA', 'EE Planta / Hl', 'EE Elaboracion / Hl', 'EE Bodega / Hl', 'EE Cocina / Hl', 'EE Envasado / Hl', 'EE Servicios / Hl', 'EE Frio / Hl', 'EE Aire / Hl', 'EE CO2 / Hl', 'EE Agua / Hl', 'Agua Planta / Hl', 'Agua Elab / Hl', 'Agua Bodega / Hl', 'Agua Cocina / Hl', 'Agua Envas / Hl', 'Agua Planta de Agua/Hl', 'Produccion Agua / Hl', 'ET Planta / Hl', 'ET Elab/Hl', 'ET Bodega/Hl', 'ET Cocina/Hl', 'ET Envasado/Hl', 'Aire Planta / Hl', 'Aire Elaboracion / Hl', 'Aire Cocina / Hl', 'Aire Bodega / Hl', 'Aire Envasado / Hl', 'CO 2 / Hl', 'Hl de Mosto', 'Cocimientos Diarios', 'Planta (Kw)', 'Elaboracion (Kw)', 'Bodega (Kw)', 'Cocina (Kw)', 'Envasado (Kw)', 'Servicios (Kw)', 'Aire (Kw)', 'Calderas (Kw)', 'Efluentes (Kw)', 'Frio (Kw)', 'Prod Agua (Kw)', 'KW CO2', 'KW Enfluentes Coc', 'KW Enfluente Efl', 'KW Enfluentes Hidr', 'Kw Compresores Aire', 'Agua Planta (Hl)', 'Agua Elaboracion (Hl)', 'Agua Bodega (Hl)', 'Agua Cocina (Hl)', 'Agua Dilucion (Hl)', 'Agua Envasado (Hl)', 'Agua Servic

# Feature enginering de preprocesamiento
Agregar nuevas variables que consideremos necesarias:
- temperaturas diarias en la zona
- variables temporales (estacion, dia)


In [13]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

Note: you may need to restart the kernel to use updated packages.


c:\Users\franc\Desktop\MostoElMostro\MostoElMostro\.venv\Scripts\python.exe: No module named pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\franc\Desktop\MostoElMostro\MostoElMostro\.venv\Scripts\python.exe: No module named pip


In [14]:
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# El cambio clave está en esta URL
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 32.5672,
    "longitude": -116.6251,
    "start_date": "2020-01-01",
    "end_date": "2023-12-31",
    "hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data=hourly_data)


hourly_dataframe

,date,temperature_2m
0,2020-01-01 00:00:00+00:00,13.594001
1,2020-01-01 01:00:00+00:00,10.694000
2,2020-01-01 02:00:00+00:00,9.344001
3,2020-01-01 03:00:00+00:00,7.894000
4,2020-01-01 04:00:00+00:00,7.694000
...,...,...
35059,2023-12-31 19:00:00+00:00,12.594001
35060,2023-12-31 20:00:00+00:00,13.344001
35061,2023-12-31 21:00:00+00:00,13.344001
35062,2023-12-31 22:00:00+00:00,13.294001


In [15]:
obj.redondear_horarios()
obj.add_estacion()
obj.add_temp_y_dia(hourly_dataframe)

✅ Columna 'ESTACION' agregada correctamente.


# Construyendo el dataset final
Nuestro datasets tiene datos temporales pero por tupla (fecha hora) hay multiples observaciones. Queremos promediar estas para ver si son series temporales pero queremos verificar que las variables categoricas no sean distintas entre los instantes (fecha hora). Entonces Vamos a verificar esto mismo con la salvedad de que la diferencia sean entre nan y otro valor.

La unica columna que genera una diferencia es Ultimo Dato del Dia. La obviamos y juntamos los datos usando las medias de las observaciones (no promedios para no ser sensibles a atipicos).

In [16]:
# ======================================================
# --- EJECUCIÓN DEL SCRIPT DE VERIFICACIÓN DE CONSISTENCIA ---
# ======================================================

# Asumimos que 'obj' es una instancia de tu clase 'make_dataset'
# y que 'cols_binarias_numericas' es la lista de columnas que ya definiste.

print("\n--- Verificando consistencia de categorías binarias antes de agrupar ---")

# 3. Ejecutar el método
# (Llamamos al método desde el objeto 'obj')
hay_problemas, grupos_problematicos = obj.verificar_consistencia_binaria(cols_binarias_numericas)

# 4. Mostrar los resultados si los hay
if hay_problemas:
    print("\n--- Detalle de Inconsistencias ---")
    
    # Mostramos los primeros 5 grupos problemáticos como ejemplo
    count = 0
    for (dia, hora), columnas in grupos_problematicos.items():
        if count < 5:
            print(f"En ({dia}, {hora}), las siguientes columnas varían: {columnas}")
        count += 1
    if count >= 5:
        print(f"... y {len(grupos_problematicos) - 5} más.")
else:
    print("   (Se puede proceder con la agrupación de forma segura)")


--- Verificando consistencia de categorías binarias antes de agrupar ---
--- Iniciando verificación de consistencia en tuplas (DIA, HORA) duplicadas ---

Revisados 29701 grupos ('DIA', 'HORA') únicos.
Se encontraron 29700 grupos con filas duplicadas.
✅ ¡Éxito! Todas las tuplas (DIA, HORA) duplicadas son consistentes en las columnas binarias.
   (Se puede proceder con la agrupación de forma segura)


In [17]:
# ======================================================
# --- EJECUCIÓN DEL SCRIPT DE AGRUPACIÓN ---
# ======================================================
# Asumimos que 'obj' es una instancia existente de tu clase 'make_dataset'

# 1. Definir las columnas que NO se deben promediar

lista_texto=[]


# 3. Llamar al método de la clase
# Esto modificará el DataFrame 'obj.df' internamente
obj.agrupar_por_media(cols_binarias_numericas, lista_texto)

# 4. Verificar el resultado
print(f"\nDimensiones finales del obj.df: {obj.df.shape}")
print(obj.df.head())

--- Iniciando agrupación por (DIA) usando la media (Promedio Diario) ---
Agrupando 79 columnas por (DIA)...

✅ Agrupación Diaria completada.
   Filas reducidas de 658797 a 1190.

Dimensiones finales del obj.df: (1190, 80)
         DIA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  \
0 2020-07-01      198.550355            14.944947       21.898397   
1 2020-07-02        7.332630             0.648068        0.725053   
2 2020-07-03        8.198708             0.756267        0.776495   
3 2020-07-04        4.550079             0.417806        0.355416   
4 2020-07-05        5.627349             0.491721        0.441206   

   EE Cocina / Hl  EE Envasado / Hl  EE Servicios / Hl  EE Frio / Hl  \
0        0.000000          1.905891         173.274186    104.618915   
1        0.522019          2.414053           5.041896      3.106550   
2        0.225265          1.869990           5.344107      3.559398   
3        0.243756          1.305393           2.621910      1.545884   
4  

Ya construido este dataset final, lo almacenamos.

In [18]:
import os
output_folder = 'dataset_FE'
output_filename = 'dataset_limpio.csv' 

# Esto crea la ruta correcta: 'dataset_FE/dataset_limpio.csv'
full_path = os.path.join(output_folder, output_filename)

# --- 2. Crear la carpeta si no existe ---
try:
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Carpeta creada en: {output_folder}")
    else:
        print(f"La carpeta '{output_folder}' ya existe.")

    # --- 3. Guardar el DataFrame ---
    
    # Usamos index=False para evitar que se guarde el índice de pandas
    # como una columna extra 'Unnamed: 0' en el CSV.
    # Usamos sep=';' y decimal='.' para buena compatibilidad con Excel.
    
    obj.df.to_csv(full_path, index=False, sep=';', decimal='.') 
    
    print(f"✅ DataFrame guardado exitosamente en: {full_path}")

except Exception as e:
    print(f"❌ Error al guardar el archivo: {e}")

Carpeta creada en: dataset_FE
✅ DataFrame guardado exitosamente en: dataset_FE\dataset_limpio.csv
